Retrieve Dataset

Imports

In [1]:
import os
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTENC
from pathlib import Path

In [2]:
dataset_path = Path('.') / 'datasets' 
train_pca_df_str = dataset_path
name = 'train_pca_df.csv'
path = dataset_path / name
test_path_name = 'test_pca_df.csv'
test_path = dataset_path / test_path_name

In [3]:
def load_dataset(name):
    path = dataset_path / name
    df = pd.read_csv(path)
    return df

## **Loading of datasets**


In [14]:
train_data = pd.read_csv(path, header=0, index_col=0)

In [15]:
train_data.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,Hinselmann
0,10.934935,-2.435187,-3.563541,-2.361388,-1.747570,-0.860913,0.842583,-0.177998,-0.130178,0
1,-1.861014,-1.402219,2.678127,1.494547,0.074622,-0.258752,0.127340,-1.081443,-0.248821,0
2,-10.672644,0.248321,-0.530190,0.034820,-0.139891,0.554827,-0.396888,0.192537,1.314509,0
3,-2.143051,11.331544,7.036601,0.891039,-0.523916,5.472790,0.215044,1.010643,-0.269921,0
4,-6.596134,-0.634044,-1.510566,1.616166,0.251334,-0.965376,0.455048,0.402326,-0.238877,0


In [19]:
test_data = pd.read_csv(test_path, header=0, index_col=0)

In [20]:
test_data.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,Hinselmann
0,-9.688302,0.041771,-0.954737,-0.055185,0.031039,-1.307739,0.522938,-0.485515,0.954248,0
1,-0.377942,-1.222654,-0.170815,-0.428940,-0.598516,-0.586628,0.369406,-1.633167,-0.247641,0
2,-12.907189,0.680422,-0.150184,-1.423120,-0.226828,-0.545328,0.236120,0.528557,-0.210510,0
3,-10.973954,0.448199,-0.385614,-1.677524,-0.388715,-0.583376,0.310400,0.762001,-0.195428,0
4,-8.599631,-0.221915,-1.305174,0.714071,0.142525,-1.168853,0.501596,-0.531533,-0.232920,0


In [21]:
test_data_X = test_data.drop(['Hinselmann'], axis=1)
test_data_Y = test_data[['Hinselmann']]

In [22]:
test_data_X.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9
0,-9.688302,0.041771,-0.954737,-0.055185,0.031039,-1.307739,0.522938,-0.485515,0.954248
1,-0.377942,-1.222654,-0.170815,-0.428940,-0.598516,-0.586628,0.369406,-1.633167,-0.247641
2,-12.907189,0.680422,-0.150184,-1.423120,-0.226828,-0.545328,0.236120,0.528557,-0.210510
3,-10.973954,0.448199,-0.385614,-1.677524,-0.388715,-0.583376,0.310400,0.762001,-0.195428
4,-8.599631,-0.221915,-1.305174,0.714071,0.142525,-1.168853,0.501596,-0.531533,-0.232920


In [23]:
binary_categories = ['Smokes', 'Hormonal Contraceptives', 'IUD', 'STDs', 'STDs:condylomatosis', 'STDs:cervical condylomatosis',
                     'STDs:vaginal condylomatosis', 'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis', 'STDs:pelvic inflammatory disease',
                     'STDs:genital herpes', 'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV', 'STDs:Hepatitis B', 'STDs:HPV', 'Dx:Cancer',
                     'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller', 'Citology', 'Biopsy']
continuous_categories = ['Age', 'Number of sexual partners', 'First sexual intercourse', 'Num of pregnancies', 'Smokes (years)', 
                         'Smokes (packs/year)', 'Hormonal Contraceptives (years)', 'IUD (years)', 'STDs (number)', 'STDs: Number of diagnosis']

In [24]:
X = train_data.drop(['Hinselmann'], axis=1)
Y = train_data[['Hinselmann']]

In [25]:
X.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9
0,10.934935,-2.435187,-3.563541,-2.361388,-1.747570,-0.860913,0.842583,-0.177998,-0.130178
1,-1.861014,-1.402219,2.678127,1.494547,0.074622,-0.258752,0.127340,-1.081443,-0.248821
2,-10.672644,0.248321,-0.530190,0.034820,-0.139891,0.554827,-0.396888,0.192537,1.314509
3,-2.143051,11.331544,7.036601,0.891039,-0.523916,5.472790,0.215044,1.010643,-0.269921
4,-6.596134,-0.634044,-1.510566,1.616166,0.251334,-0.965376,0.455048,0.402326,-0.238877


In [26]:
Y.head()

,Hinselmann
0,0
1,0
2,0
3,0
4,0


# **Random Forest Classifier**

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
classifier = RandomForestClassifier()
classifier_parameters = {
    'n_estimators': [10, 100, 1000],
    'criterion' : ['gini', 'entropy'],
    'max_depth': list(range(1, 8)), #I'm trying with the lowes tone first #intervallo derivante dal finetuning del decision tree
    'min_samples_split': list(range(2, 9)), #intervallo derivante dal finetuning del decision tree
    'max_features': ['sqrt'],
    'random_state': [42]    
}

scoring = 'accuracy' # for now, later also recall

In [17]:
from sklearn.model_selection import GridSearchCV

In [30]:
grid_search_result = GridSearchCV(classifier, classifier_parameters, scoring=scoring)

fit_result = grid_search_result.fit(X, Y)

params = fit_result.best_params_

C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\s

C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\s

C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\s

C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\s

C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\s

...

C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\s

C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\s

C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\s

C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\s

C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\s

In [31]:
rf_best_params = params
rf_best_params

{'criterion': 'gini',
 'max_depth': 7,
 'max_features': 'sqrt',
 'min_samples_split': 5,
 'n_estimators': 100,
 'random_state': 42}

In [32]:
tuned_classifier = grid_search_result.best_estimator_
tuned_classifier

RandomForestClassifier(max_depth=7, max_features='sqrt', min_samples_split=5,
                       random_state=42)

In [33]:
# Prediction
prediction = tuned_classifier.predict(test_data_X)

In [34]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_data_Y, prediction)
print(cm)

[[169  22]
 [  8   2]]


In [18]:
from sklearn.metrics import confusion_matrix

In [19]:
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve
import sklearn

In [20]:
from joblib import dump, load

In [35]:
tn, fp, fn, tp = cm.ravel()
print(f"tp={tp}, fn={fn}, fp={fp}, tn={tn}")
test_accuracy = (tn + tp) / (tn + fp + fn + tp)
test_recall = (tp) / (tp + fn)
print(f'accuracy={test_accuracy}; recall={test_recall}')
print("-"*10)
print()
print()

tp=2, fn=8, fp=22, tn=169
accuracy=0.8507462686567164; recall=0.2
----------




In [43]:
accuracy_matrix = np.empty((1,1))
recall_matrix = np.empty((1,1))

#accuracy_matrix = np.empty((len(classifiers), len(train_datasets)))

#recall_matrix = np.empty((len(classifiers), len(train_datasets)))

In [38]:
classifier_name = 'RandomForest'
dataset_name = 'pca'

In [44]:
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve
import sklearn

test_accuracy = (tn + tp) / (tn + fp + fn + tp)
test_recall = (tp) / (tp + fn)

if scoring == 'accuracy':
    # accuracy_matrix[i, j] = test_accuracy
    accuracy_matrix = test_accuracy
elif scoring == 'recall':
    # recall_matrix[i, j] = test_recall
    recall_matrix = test_recall

fig, ax = plt.subplots(figsize=(5,5),ncols=1)
fig = plot_confusion_matrix(tuned_classifier, test_data_X, test_data_Y, labels=[1,0], ax=ax)
plt.plot()
plt.savefig(Path('.') / 'images' / 'confusion_matrix' / f'{scoring}_{classifier_name}_{dataset_name}_confusion_matrix.png')
plt.close()

fig, ax = plt.subplots(figsize=(5,5),ncols=1)
plot_roc_curve(tuned_classifier, test_data_X, test_data_Y, ax=ax)
plt.savefig(Path('.') / 'images' / 'roc' / f'{scoring}_{classifier_name}_{dataset_name}_roc.png')
plt.plot()
plt.close()

C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Xileny\Anaconda3\envs\ds_cervical_cancer\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function plot_roc_curve is deprecated; Function :func:`plot_roc_curve` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: :meth:`sklearn.metric.RocCurveDisplay.from_predictions` or :meth:`sklearn.metric.RocCurveDisplay.from_estimator`.
  warnings.warn(msg, category=FutureWarning)


In [45]:
print(accuracy_matrix)
print(recall_matrix)

0.8507462686567164
[[4.26521154e-312]]


In [46]:
test_data.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,Hinselmann
0,-9.688302,0.041771,-0.954737,-0.055185,0.031039,-1.307739,0.522938,-0.485515,0.954248,0
1,-0.377942,-1.222654,-0.170815,-0.428940,-0.598516,-0.586628,0.369406,-1.633167,-0.247641,0
2,-12.907189,0.680422,-0.150184,-1.423120,-0.226828,-0.545328,0.236120,0.528557,-0.210510,0
3,-10.973954,0.448199,-0.385614,-1.677524,-0.388715,-0.583376,0.310400,0.762001,-0.195428,0
4,-8.599631,-0.221915,-1.305174,0.714071,0.142525,-1.168853,0.501596,-0.531533,-0.232920,0


In [47]:
X.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9
0,10.934935,-2.435187,-3.563541,-2.361388,-1.747570,-0.860913,0.842583,-0.177998,-0.130178
1,-1.861014,-1.402219,2.678127,1.494547,0.074622,-0.258752,0.127340,-1.081443,-0.248821
2,-10.672644,0.248321,-0.530190,0.034820,-0.139891,0.554827,-0.396888,0.192537,1.314509
3,-2.143051,11.331544,7.036601,0.891039,-0.523916,5.472790,0.215044,1.010643,-0.269921
4,-6.596134,-0.634044,-1.510566,1.616166,0.251334,-0.965376,0.455048,0.402326,-0.238877


In [48]:
# TODO save classifier
from joblib import dump, load
dump(tuned_classifier, Path('.') / 'models' /f'{scoring}_{classifier_name}_{dataset_name}_tuned_model.joblib') 

['models\\accuracy_RandomForest_pca_tuned_model.joblib']

# **Decision tree **classifier****

In [5]:
from sklearn.tree import DecisionTreeClassifier

In [6]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [66]:
dtc_classifier = DecisionTreeClassifier()
dtc_decision_tree_parameters = {
  'max_depth': list(range(2, 10)),
  'min_samples_split': list(range(2, 10)),
  'criterion' : ['gini', 'entropy'],
  'random_state': [42]
  }

In [68]:
dtc_grid_search_result = GridSearchCV(dtc_classifier, dtc_decision_tree_parameters, scoring=scoring)

dtc_fit_result = dtc_grid_search_result.fit(X, Y)

dtc_params = dtc_fit_result.best_params_

In [69]:
dtc_best_params = dtc_params
dtc_best_params

{'criterion': 'entropy',
 'max_depth': 9,
 'min_samples_split': 2,
 'random_state': 42}

In [70]:
dtc_tuned_classifier = dtc_grid_search_result.best_estimator_
dtc_tuned_classifier

DecisionTreeClassifier(criterion='entropy', max_depth=9, random_state=42)

In [8]:
classifiers = {
    'random_forest': RandomForestClassifier(),
    'svc': SVC(),
    'decision_tree': DecisionTreeClassifier(),    
    'logistic_regression': LogisticRegression(),
    'knn': KNeighborsClassifier()
  }

In [9]:
random_forest_parameters = {
    'n_estimators': [10, 100, 1000],
    'criterion' : ['gini', 'entropy'],
    'max_depth': list(range(1, 8)), #I'm trying with the lowes tone first #intervallo derivante dal finetuning del decision tree
    'min_samples_split': list(range(2, 9)), #intervallo derivante dal finetuning del decision tree
    'max_features': ['sqrt'],
    'random_state': [42]    
}

In [10]:
svc_parameters = {
  'C': [0.1, 1, 10, 100],
  'gamma': [1, 0.1, 0.01, 0.001],
  'kernel': ['rbf'] #, 'poly', 'linear']
}

In [11]:
decision_tree_parameters = {
  'max_depth': list(range(2, 10)),
  'min_samples_split': list(range(2, 10)),
  'criterion' : ['gini', 'entropy'],
  'random_state': [42]
}

In [12]:
logistic_regression_parameters = {
    'penalty': ['l1', 'l2'],
    'random_state': [42],
    'C': [1000, 100, 10, 1, 0.1, 0.01, 0.001],
    'max_iter': [1000],
    'solver': ['liblinear']
}

In [13]:
knn_parameters = {
    'leaf_size' : list(range(1,50, 4)),
    'n_neighbors' : list(range(1,35, 4)), #1+4n is always odd
    'p': [1, 2], #power parameter for the Minkowski metric.
    'weights': ['uniform', 'distance' ],
    'n_jobs' : [-1]
}

In [23]:
classifiers_parameters = {
    'random_forest': random_forest_parameters,
    'svc': svc_parameters,
    'decision_tree': decision_tree_parameters,
    'logistic_regression': logistic_regression_parameters,
    'knn': knn_parameters
}

In [15]:
train_datasets = {
  #'with_outliers': load_dataset('ThoracicSurgery_with_outliers_normalized_train.csv'),
  'no_oversampling': load_dataset('train_data.csv'),
  #'random_oversampling': load_dataset('ThoracicSurgery_RandomOverSampler_train.csv'),
  #'smote': load_dataset('oversampled_train_data.csv'),
  'pca': load_dataset('train_pca_df.csv')
 }

test_datasets = {
  #'with_outliers': load_dataset(name= 'ThoracicSurgery_with_outliers_normalized_test.csv'),
  'no_oversampling': load_dataset('test_data.csv'),
  #'random_oversampling': load_dataset(name='ThoracicSurgery_test.csv'),
  #'smote': load_dataset('oversampled_test_data.csv'),
  'pca': load_dataset('test_pca_df.csv')
}

tuned_classifiers = {
    'random_forest': [],
    'svc': [],
    'decision_tree': [],    
    'logistic_regression': [],
    'knn': []
} 

In [16]:
accuracy_matrix = np.empty((len(classifiers), len(train_datasets)))
recall_matrix = np.empty((len(classifiers), len(train_datasets)))

In [26]:
for scoring in ['accuracy', 'recall']:
    for j, dataset in enumerate(train_datasets.keys()):
        for i, classifier in enumerate(classifiers.keys()):
            print("Classifier: ", classifier)

            train_df = train_datasets[dataset]
            test_df = test_datasets[dataset]

            x_labels = train_df.keys()[0:-1]
            y_label = train_df.keys()[-1]

            X = train_df[x_labels]
            Y = train_df[y_label]
            X_test = test_df[x_labels]
            Y_test = test_df[y_label]

            classifier_name = f'{scoring}_{classifier}_{dataset}_tuned_model.joblib'
            classifier_path =  Path('.') / 'tuned_classifiers' / classifier_name
            print("Checking if file {} exists".format(classifier_path))
            if classifier_path.exists():
                tuned_classifier = load(str(classifier_path))
                print("{} exists".format(classifier_path))
            else:
                print("{} does not exist".format(classifier_path))
                selected_classifier = classifiers[classifier]
                parametrers = classifiers_parameters[classifier]
                grid_search_result = GridSearchCV(selected_classifier, parametrers, scoring=scoring)

                fit_result = grid_search_result.fit(X, Y)

                params = fit_result.best_params_

                print("-"*10)
                print("Best parameters using the scoring {} for classifier {} on the dataset {} : ".format(scoring, classifier, dataset))
                print(params)

                tuned_classifier = grid_search_result.best_estimator_

                #saving tuned classifier
                dump(tuned_classifier, Path('.') / 'models' /f'{scoring}_{classifier}_{dataset}_tuned_model.joblib') 

            print("Appending classifier at position: ", len(tuned_classifiers[classifier]))
            tuned_classifiers[classifier].append(tuned_classifier)

            prediction = tuned_classifier.predict(X_test)
            cm = confusion_matrix(Y_test, prediction)
            print(cm)

            tn, fp, fn, tp = cm.ravel()
            print(f"tp={tp}, fn={fn}, fp={fp}, tn={tn}")
            test_accuracy = (tn + tp) / (tn + fp + fn + tp)
            test_recall = (tp) / (tp + fn)
            print(f'accuracy={test_accuracy}; recall={test_recall}')
            print("-"*10)
            print()
            print()


            test_accuracy = (tn + tp) / (tn + fp + fn + tp)
            test_recall = (tp) / (tp + fn)

            if scoring == 'accuracy':
                accuracy_matrix[i, j] = test_accuracy
            elif scoring == 'recall':
                recall_matrix[i, j] = test_recall

            fig, ax = plt.subplots(figsize=(5,5),ncols=1)
            fig = plot_confusion_matrix(tuned_classifier, X_test, Y_test, labels=[1,0], ax=ax)
            plt.savefig(Path('.') / 'images' / 'confusion_matrix' / f'{scoring}_{classifier}_{dataset}_confusion_matrix.png')
            plt.close()

            fig, ax = plt.subplots(figsize=(5,5),ncols=1)
            sklearn.metrics.plot_roc_curve(tuned_classifier, X_test, Y_test, ax=ax)
            plt.savefig(Path('.') / 'images' / 'roc' / f'{scoring}_{classifier}_{dataset}_roc.png')
            plt.close()

Classifier:  random_forest
Checking if file tuned_classifiers\accuracy_random_forest_no_oversampling_tuned_model.joblib exists
tuned_classifiers\accuracy_random_forest_no_oversampling_tuned_model.joblib does not exist


KeyboardInterrupt: 

In [67]:
print(accuracy_matrix)
print(recall_matrix)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
